# Required Packages

In [142]:
# pip install getFamaFrenchFactors

In [143]:
# pip install yfinance

In [144]:
# pip install eventstudy

In [145]:
# pip install tensorflow

In [146]:
# pip install --force-reinstall transformers

# 3000 News from Professor

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import datetime as dt
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

/Users/niko/.pyenv/versions/3.9.7/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#from google.colab import files
 
#Upload here the News3k.csv

#uploaded = files.upload()

In [5]:
news3k = pd.read_csv("CryptoLin.csv", encoding= 'unicode_escape')
pd.set_option('display.max_colwidth', -1)
# news3k.head()

/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/1443087665.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
# news3k

In [10]:
news3k["date_parsed"]=pd.to_datetime(news3k["date"])

In [11]:
# news3k.dtypes

467 news out of the 3000 are from 2022

---
# Fama French Three Factor
<p>&nbsp;</p>
https://medium.com/the-handbook-of-coding-in-finance/estimating-stock-returns-with-fama-french-three-factor-model-in-python-1a98e3936859
<p>&nbsp;</p>
https://github.com/teobeeguan/Python-For-Finance/blob/main/Portfolio%20Management/Fama_French_Three_Factor.py


In [39]:
news3k["date_parsed"].min()

Timestamp('2018-07-30 00:00:00')

In [13]:
ticker = "BTC-USD" # add here the place holder for the stock input of the user
start = "2014-12-01"
end = "2022-02-01"

In [14]:
stock_data = yf.download(ticker, start, end, adjusted=True)
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

[*********************100%***********************]  1 of 1 completed


In [15]:
# stock_data.head()

In [147]:
# ff3_monthly

### Expected Monthly Return

In [17]:
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = "Month_Rtn"
ff_data = ff3_monthly.merge(stock_returns,on='Date')

In [18]:
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
# print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

In [19]:
rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()

expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12

### Expected Daily Return

In [20]:
"""
Created on Tue Aug 31 15:11:15 2021
@author: Teo Bee Guan
"""
stock_data = yf.download(ticker, start, end, adjusted=True)

[*********************100%***********************]  1 of 1 completed


In [35]:
stock_returns.name = "Day_Rtn"

In [22]:
#from google.colab import files

#Upload FF3_daily.csv

#uploaded = files.upload()

In [23]:
ff3_daily=pd.read_csv("FF3_daily.csv",parse_dates=['date'])
ff3_daily.rename(columns={"date": 'Date'}, inplace=True)
ff3_daily.set_index('Date', inplace=True)

In [24]:
ff_data = ff3_daily.merge(stock_returns,on='Date')

In [25]:
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Day_Rtn'] - ff_data['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
# print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

In [26]:
rf = ff_data['RF'].mean()
market_premium = ff3_daily['Mkt-RF'].mean()
size_premium = ff3_daily['SMB'].mean()
value_premium = ff3_daily['HML'].mean()

expected_daily_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 

print("Expected return for:",ticker)
print("Daily return: " + str(round(expected_daily_return,5)*100)+'%')
print("Monthly return: " + str(round(expected_monthly_return,5)*100)+'%')
print("Yearly return: " + str(round(expected_yearly_return,5)*100)+'%')

Expected return for: BTC-USD
Daily return: 0.219%
Monthly return: 1.004%
Yearly return: 12.046%


# Event Study

ff3_dailay source: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

In [27]:
ff3_daily=pd.read_csv("FF3_daily.csv")
ff3_daily.tail()

,date,Mkt-RF,SMB,HML,RF
25143,20211227,1.22,-0.09,0.28,0.0
25144,20211228,-0.27,-0.59,0.80,0.0
25145,20211229,0.06,-0.06,0.15,0.0
25146,20211230,-0.15,0.10,-0.38,0.0
25147,20211231,-0.31,0.06,0.53,0.0


In [36]:
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

# stock_data = pdr.get_data_yahoo(ticker, start, end)

import statsmodels.api as sm
import getFamaFrenchFactors as gff

stock_data = yf.download(ticker, start, end, adjusted=True)

'''muting this code from professor

ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'date'}, inplace=True)
'''
#ff3_monthly.set_index('date', inplace=True)


## Renaming columns and formating data to later use by es.Single.import_FamaFrench

#ff3_monthly.reset_index(level=0, inplace=True)
#ff3_monthly.columns = ['date', 'Mkt-RF', 'SMB', 'HML', 'RF']
#ff3_monthly.to_csv("test_datetime_format.csv", index=False, date_format='%Y%m%d')

'''muting this code from professor

ff3_monthly.to_csv("{}_famafrench.csv".format(ticker), index=False, date_format='%Y%m%d')
'''
ff3_daily.to_csv("{}_famafrench.csv".format(ticker), index=False, date_format='%Y%m%d')



## Renaming columns and formating data to later use by es.Single.import_FamaFrench

#stock_data = pdr.get_data_yahoo(ticker, start="2017-12-01", end="2022-02-01")

stock_data[ticker] = stock_data["Adj Close"].pct_change()
stock_data = stock_data.dropna()
stock_data.reset_index(level=0, inplace=True)
stock_data = stock_data[['Date',ticker]].copy()
stock_data.columns = ['date',ticker]
stock_data.to_csv("{}_returns.csv".format(ticker), index=False, date_format='%Y-%m-%d')


## Importing Fama French 3 Factors and Stock Return

import eventstudy as es
# print(es.__version__)
from eventstudy import excelExporter

import numpy as np
import matplotlib.pyplot as plt


es.Single.import_FamaFrench("{}_famafrench.csv".format(ticker))
es.Single.import_returns("{}_returns.csv".format(ticker))

event = es.Single.FamaFrench_3factor(
    security_ticker = ticker,
    event_date = np.datetime64('2020-12-28'),
    event_window = (-2,+4), 
    estimation_size = 100,
    buffer_size = 30
)

# event.plot(AR=True)
# plt.show()

[*********************100%***********************]  1 of 1 completed


In [37]:
event.AR

array([ 0.06980493, -0.0118392 ,  0.02529761,  0.00250216,  0.05144785,
       -0.00028216,  0.00835475])

In [38]:
event.CAR

array([0.06980493, 0.05796573, 0.08326334, 0.0857655 , 0.13721335,
       0.13693119, 0.14528594])

In [40]:
news3k["date_parsed"].nunique()

398

In [41]:
event.results(decimals=[3,5,3,5,2,2])

,AR,Std. E. AR,CAR,Std. E. CAR,T-stat,P-value
-2,0.070,0.02565,0.07 **,0.02565,2.72,0.01
-1,-0.012,0.02565,0.058,0.03628,1.60,0.11
0,0.025,0.02565,0.083 *,0.04443,1.87,0.06
1,0.003,0.02565,0.086,0.05131,1.67,0.10
2,0.051,0.02565,0.137 **,0.05736,2.39,0.02
3,-0.000,0.02565,0.137 **,0.06284,2.18,0.03
4,0.008,0.02565,0.145 **,0.06787,2.14,0.03


---
### Loop on dates for event study

In [42]:
news3k["date_parsed"].drop_duplicates()

0      2022-01-25
12     2022-01-24
21     2022-01-23
25     2022-01-22
29     2022-01-21
          ...    
2677   2020-05-05
2678   2020-05-01
2679   2020-04-18
2680   2020-04-10
2681   2018-07-30
Name: date_parsed, Length: 398, dtype: datetime64[ns]

In [43]:
dates_for_eventstudy = news3k["date_parsed"].drop_duplicates().sort_values()
dates_for_eventstudy.count()

398

In [44]:
# dates_for_eventstudy.min()
# Timestamp('2018-07-30 00:00:00')


Timestamp('2018-07-30 00:00:00')

In [45]:
# dates_for_eventstudy.max()
# Timestamp('2022-01-25 00:00:00')


Timestamp('2022-01-25 00:00:00')

In [46]:
i = 1
listAR=[]
dates=[]
for date in dates_for_eventstudy:
    #print(date)
    try:
        event = es.Single.FamaFrench_3factor(
            security_ticker = ticker,
            event_date = np.datetime64(date),
            event_window = (-2,+4), 
            estimation_size = 100,
            buffer_size = 30
        )

        #event.plot(AR=True)
        #plt.show()
        listAR.append(event.AR)
        dates.append(date)
        
    except:
        #print("Date error")
        list_nans = []
        for nans in range(event.event_window_size):
            list_nans.append(np.nan)
        listAR.append(list_nans)
        dates.append(date)
        
    #df_test = pd.DataFrame({'date':dates_for_eventstudy, 'value':list1})
    

  #  if i == 4:
  #      break
    
  #  i += 1

columns_ar=[]
for i in range(event.event_window[0],event.event_window[1]+1):
    columns_ar.append("AR"+str(i))

df_AR = pd.DataFrame(listAR, columns=columns_ar)
df_test2 = pd.DataFrame({'date':dates}).join(df_AR)

In [148]:
# df_test2

In [149]:
df_test3 = df_test2.dropna()
df_test3["date"] = pd.to_datetime(df_test3["date"])
# df_test3

/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/409418028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test3["date"] = pd.to_datetime(df_test3["date"])


In [49]:
'''Relevant means that at least one day had AR higher than expected montly returns * 6 (Semester)'''
n = 6  #6 is for semester

relevant_matrix_pos = df_test2.iloc[:,1:len(df_test3.columns)] > expected_monthly_return*n 
relevant_matrix_neg = df_test2.iloc[:,1:len(df_test3.columns)] < -expected_monthly_return*n

#BOOLEAN
df_test3["Relevant_pos"] = relevant_matrix_pos[list(relevant_matrix_pos.columns)].any(axis=1).astype(int)
df_test3["Relevant_neg"] = -1*(relevant_matrix_neg[list(relevant_matrix_neg.columns)].any(axis=1).astype(int))


df_test3.tail()

/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/4263963401.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test3["Relevant_pos"] = relevant_matrix_pos[list(relevant_matrix_pos.columns)].any(axis=1).astype(int)
/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/4263963401.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test3["Relevant_neg"] = -1*(relevant_matrix_neg[list(relevant_matrix_neg.columns)].any(axis=1).astype(int))


,date,AR-2,AR-1,AR0,AR1,AR2,AR3,AR4,Relevant_pos,Relevant_neg
365,2021-12-23,0.053887,-0.007328,0.042052,0.002785,-0.010408,0.004789,-0.009500,0,0
366,2021-12-24,-0.005813,0.043771,0.003097,-0.011414,0.005128,-0.008136,-0.061406,0,-1
367,2021-12-25,0.041912,-0.001492,-0.010867,0.004690,-0.005885,-0.061998,-0.027376,0,-1
368,2021-12-26,0.002828,-0.008574,0.015485,-0.004025,-0.062491,-0.030989,0.012307,0,-1
369,2021-12-27,-0.012521,0.003504,-0.009243,-0.062231,-0.026660,0.014263,-0.020724,0,-1


In [50]:
print("Threshold:", expected_monthly_return*n)

Threshold: 0.06023023278871147


In [51]:
#Days with high volatility (relevant positive and negative)
# df_test3[(df_test3["Relevant_neg"] == -1) & (df_test3["Relevant_pos"] == 1)]

,date,AR-2,AR-1,AR0,AR1,AR2,AR3,AR4,Relevant_pos,Relevant_neg
25,2020-11-26,0.023244,-0.019399,-0.081413,-0.005158,0.025960,0.018030,0.078922,1,-1
26,2020-11-28,-0.090174,-0.008776,0.031673,0.028016,0.077014,-0.048362,0.014640,1,-1
32,2021-01-07,0.053958,0.060173,0.063564,0.035601,-0.022412,-0.058337,-0.076203,1,-1
33,2021-01-08,0.060265,0.063714,0.035699,-0.022405,-0.058342,-0.076084,-0.056106,1,-1
34,2021-01-09,0.063802,0.035767,-0.022304,-0.058161,-0.076079,-0.055774,0.099873,1,-1
...,...,...,...,...,...,...,...,...,...,...
273,2021-09-20,0.020703,-0.024705,-0.087675,-0.053367,0.071113,0.030326,-0.041116,1,-1
274,2021-09-21,-0.025001,-0.088398,-0.053978,0.071635,0.031240,-0.040864,-0.000940,1,-1
275,2021-09-22,-0.086050,-0.056730,0.068820,0.028382,-0.041583,0.000786,0.025880,1,-1
300,2021-10-17,0.094219,-0.006710,0.012133,0.013849,0.034213,0.029002,-0.067220,1,-1


In [52]:
# df_test3[df_test3["Relevant_neg"] == -1].count()

date            162
AR-2            162
AR-1            162
AR0             162
AR1             162
AR2             162
AR3             162
AR4             162
Relevant_pos    162
Relevant_neg    162
dtype: int64

In [53]:
# df_test3[df_test3["Relevant_pos"] == 1].count()

date            149
AR-2            149
AR-1            149
AR0             149
AR1             149
AR2             149
AR3             149
AR4             149
Relevant_pos    149
Relevant_neg    149
dtype: int64

In [54]:
# df_test3.date[(df_test3["Relevant_pos"]==1) | (df_test3["Relevant_pos"]==-1)]

2     2020-04-18
3     2020-05-01
4     2020-05-05
21    2020-11-02
25    2020-11-26
         ...    
319   2021-11-06
320   2021-11-08
321   2021-11-09
322   2021-11-10
340   2021-11-28
Name: date, Length: 149, dtype: datetime64[ns]

---
### Appending News with event study

In [55]:
news3k["date_parsed"]=pd.to_datetime(news3k['date_parsed'])


In [112]:
news3k_relevant = pd.merge(news3k,df_test3, left_on ="date_parsed", right_on="date").drop(columns=["date_parsed"],axis=1)

In [113]:
news3k_relevant.columns

Index(['id', 'date_x', 'news', 'final_manual_labelling', 'text_span', 'date_y',
       'AR-2', 'AR-1', 'AR0', 'AR1', 'AR2', 'AR3', 'AR4', 'Relevant_pos',
       'Relevant_neg'],
      dtype='object')

In [114]:
news3k_relevant_1 = news3k_relevant[(news3k_relevant["Relevant_pos"]==1) | (news3k_relevant["Relevant_pos"]==-1)]
# news3k_relevant_1.tail()

In [115]:
news3k_relevant_1['mean_AR'] = news3k_relevant_1.mean(axis=1)

/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/2247192074.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  news3k_relevant_1['mean_AR'] = news3k_relevant_1.mean(axis=1)
/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/2247192074.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  news3k_relevant_1['mean_AR'] = news3k_relevant_1.mean(axis=1)
/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/2247192074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [119]:
news3k_relevant_1.head()

,id,date_x,news,final_manual_labelling,text_span,date_y,AR-2,AR-1,AR0,AR1,AR2,AR3,AR4,Relevant_pos,Relevant_neg,mean_AR
255,495,11/28/2021,Bank of France official: Existing regulatory frameworks ÂconstrainedÂ by DeFi characteristics,-1,{annotator1_id:10;annotator1_label:-1; annotator2_id:5; annotator2_label2:-1; annotator3_id:7;annotator3_label:-1},2021-11-28,-0.068657,-0.008857,0.062691,-0.034344,-0.039142,0.014051,-0.029041,1,-1,44.899700
256,496,11/28/2021,Decentralized exchanges near $100 billion in monthly volume for November,1,{annotator1_id:21;annotator1_label:1; annotator2_id:2; annotator2_label2:1; annotator3_id:4;annotator3_label:1},2021-11-28,-0.068657,-0.008857,0.062691,-0.034344,-0.039142,0.014051,-0.029041,1,-1,45.172427
257,497,11/28/2021,"QANplatform (QANX), the quantum-resistant blockchain platform launches its CEX listing journey on Gate.io",1,{annotator1_id:15;annotator1_label:1; annotator2_id:23; annotator2_label2:1; annotator3_id:24;annotator3_label:1},2021-11-28,-0.068657,-0.008857,0.062691,-0.034344,-0.039142,0.014051,-0.029041,1,-1,45.263337
387,627,11/10/2021,Axie Infinity co-founder joins early-stage token fund 1kx as venture partner,0,{annotator1_id:22;annotator1_label:0; annotator2_id:9; annotator2_label2:0; annotator3_id:4;annotator3_label:0},2021-11-10,0.061993,-0.013972,-0.034900,-0.005808,-0.016713,-0.000151,0.010776,1,0,57.091020
388,628,11/10/2021,Country Music Association to release NFT collection for its latest award show,0,{annotator1_id:15;annotator1_label:0; annotator2_id:2; annotator2_label2:0; annotator3_id:17;annotator3_label:0},2021-11-10,0.061993,-0.013972,-0.034900,-0.005808,-0.016713,-0.000151,0.010776,1,0,57.181930


In [139]:
# news3k_relevant_1

mean_AR = news3k_relevant_1.mean_AR

# construct the probability density function
# def normal_dist(x , mean , sd):
#     prob_density = (np.pi*sd) * np.exp(-0.5*((x-mean)/sd)**2)
#     return prob_density

#Calculate mean and Standard deviation.
mean = np.mean(mean_AR)
sd = np.std(mean_AR)

# pdf = normal_dist(mean_AR,mean,sd)

# get lower and upper limit for the normal distribution
upper_limit = mean+sd
lower_limit = mean-sd

# plot the normal distribution
# plt.plot(mean_AR,pdf , color = 'red')
# plt.xlabel('Data points')
# plt.ylabel('Probability Density')


news3k_relevant_1["strength"] = np.where((news3k_relevant_1["mean_AR"] < upper_limit) & (news3k_relevant_1["mean_AR"] > lower_limit), 1, 0)
news3k_relevant_1["strength"].value_counts()

/var/folders/m4/xkrhq1cj3gq0qvjhg5y8bh480000gq/T/ipykernel_55601/3886233552.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news3k_relevant_1["strength"] = np.where((news3k_relevant_1["mean_AR"] < upper_limit) & (news3k_relevant_1["mean_AR"] > lower_limit), 1, 0)


0    515
1    479
Name: strength, dtype: int64

In [150]:
to_export = news3k_relevant_1[['news','strength']]
to_export.to_csv('/Users/niko/Downloads/GROUP_PROJECT_CRYPTO_CURATED_DATABASE/strength_score_with_news')

In [60]:
#More news in 2021 than other years, still growing
# pd.DataFrame(news3k_relevant.groupby(news3k_relevant['date_x']).size()).plot()

In [71]:
# pd.DataFrame(news3k_relevant_1.groupby(news3k_relevant_1['date_x']).size()).plot()

In [72]:
# pd.DataFrame(news3k.groupby(news3k['date_parsed']).size()).plot()